In [1]:
import jax.numpy as jnp
import numpy as np
import meshplot as mp
from mesh_processing import precompute_mesh_attributes, compute_mesh_attributes, compute_laplacian_crane, compute_laplacian_sorkine

In [2]:
DATA = './meshes/'
meshfile = 'stanford_bunny_original.obj'
#meshfile = 'suzanne.obj'

V, F, NF, E, NV, N, A, VF, D, El, DEl, DE, DA = precompute_mesh_attributes(DATA, meshfile)

d = mp.plot(np.array(V), np.array(F), return_plot=True, shading={'wireframe': True})

/home/max/anaconda3/lib/python3.7/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


Loading in mesh: 3.6495230197906494 s
edge connectivity + lens: 0.24063444137573242 s
vertex neighbourhood connectivity: 0.07940125465393066 s
face neighbourhood connectivity: 0.0579371452331543 s
normals + area: 0.0936582088470459 s
dual points: 0.10742974281311035 s
vertex - face adjacency: 0.5072052478790283 s
dual edge computation: 12.426661252975464 s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016800…

In [3]:
L = compute_laplacian_crane(V, E, El, DEl)

In [4]:
# Set boundary conditions:
boundary_high = [vi for vi, v in enumerate(V) if v[1] > np.mean(V[:,1]) + np.mean(V[:, 1]) * 0.75]
boundary_low = [vi for vi, v in enumerate(V) if v[0] > np.mean(V[:,0]) - np.mean(V[:, 0]) * 2.8]
f = np.zeros(V.shape[0]); c = np.zeros((V.shape[0], 3))
f[boundary_high] = -1.0
f[boundary_low] = 1.0
c[boundary_high] = [1.0, 0.0, 0.0]
c[boundary_low] = [0.0, 0.0, 1.0]
mp.plot(np.array(V), np.array(F), return_plot=True, c=c, shading={'wireframe': False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016800…

In [5]:
c_high = np.array([1.0, 0.0, 0.0])
c_low = np.array([0.0, 0.0, 1.0])

h_c = (f - 1.0) / 2.
l_c = (f + 1.0) / 2.
c = h_c + l_c
mp.plot(np.array(V), np.array(F), return_plot=True, c=c, shading={'wireframe': False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016800…

In [6]:
for vi in boundary_high:
    L[vi, :] = 0
    L[vi, vi] = 1
for vi in boundary_low:
    L[vi, :] = 0
    L[vi, vi] = 1
poisson = np.matmul(np.linalg.inv(L), f)

In [8]:
d = mp.subplot(np.array(V), np.array(F), c=c, shading={'wireframe': False}, s=[1, 2, 0])
h_c = (poisson - 1.0) / 2.
l_c = (poisson + 1.0) / 2.
c = h_c + l_c

mp.subplot(np.array(V), np.array(F), c=c, shading={'wireframe': False}, s=[1,2,1], data=d)

In [18]:
L = compute_laplacian_crane(V, E, El, DEl)
fu = np.zeros(V.shape[0])
fv = np.zeros(V.shape[0])
L[0, :] = 0
L[1, :] = 0
L[2, :] = 0
L[0, 0] = 1
L[1, 1] = 1
L[2, 2] = 1
fu[0] = 1.0
fv[0] = 0.0
fu[1] = 0.0
fv[1] = 1.0
fu[2] = 1.0
fv[2] = 1.0

u = np.matmul(np.linalg.inv(L), fu)
v = np.matmul(np.linalg.inv(L), fv)
z = np.zeros(V.shape[0])
V_ = np.column_stack([u, v, z])


In [20]:
mp.plot(np.array(V_), np.array(F), shading={'wireframe': True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…